In [46]:
import requests
from datetime import datetime
import pandas as pd
from io import StringIO
from collections import Counter
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
# Code voor het activeren van de API

def gather_metadata(org_id, count):

    # URL and headers
    url = "https://api.openraadsinformatie.nl/v1/elastic/ori_*/_search?"
    headers = {
        "Content-Type": "application/json"
    }
    
    # Request JSON data
    data = {
        "query": {
            "bool": {
                "must": [
                    {
                        "simple_query_string": {
                            "fields": ["text", "title", "description", "name"],
                            "default_operator": "or",
                            "query": "*"
                        }
                    },
                    {
                        "terms": {
                            "_index": [org_id],
                        }
                    }
                ]
            }
        },
        "size": count,
        "_source": {
            "includes": ["*"],
            "excludes": []
        },
        "from": 0,
        "sort": [
            {
                "_score": {
                    "order": "desc"
                }
            }
        ],
    }
    
    
    # Post request versturen
    response = requests.post(url, json=data, headers=headers)
    
    # Check de response
    if response.status_code == 200:
        result = response.json()
    
    else:
        print("Request failed with status code:", response.status_code)
        print("Response content:", response.text)

    print(len(result['hits']['hits']))

    all_results = {r['_id']: r for r in result['hits']['hits']}

    return all_results


In [3]:
def process_report(id, all_results, gemeentenaam):
    json_data = dict()
    
    data = all_results[id]


    json_data['id'] = id
    try:
        json_data['dc_title'] = data['_source']['name']   
    except KeyError:
        return {}
    json_data['dc_source'] = f"https://{gemeentenaam}.bestuurlijkeinformatie.nl/Reports/Item/{data['_source']['was_generated_by']['original_identifier']}"
    json_data['dc_type'] = data['_source']['@type']   
    json_data['foi_classificaties'] = data['_source']['classification']
    json_data['dc_publisher'] = all_results[data['_source']['has_organization_name']]['_source']['name']
    try:
        json_data['dc_description'] = data['_source']['description']
    except KeyError:
        pass

    
    try:
        json_data['foi_publishedDate'] = datetime.fromisoformat(data['_source']['start_date']).strftime('%Y-%m-%d')

    except KeyError:
        json_data['foi_publishedDate'] = datetime.fromisoformat(data['_source']['was_generated_by']['started_at_time']).strftime('%Y-%m-%d')

    json_data['dc_date_year'] = json_data['foi_publishedDate'][:4]

    foi_files = []
    
    if 'attachment' in data['_source']:

        attachments = data['_source']['attachment']
        if isinstance(attachments, str):
            attachments = [attachments]

    
        for a in attachments:
            try:
                data = all_results[a]
                temp_data = dict()
                temp_data['dc_title'] = data['_source']['name']
                temp_data['dc_type'] = data['_source']['@type']
                temp_data['dc_source'] = data['_source']['original_url']
                temp_data['dc_format'] = data['_source']['content_type']
                bodytext = [txt for txt in data['_source']['text'] if txt != '\x0c']
                temp_data['foi_bodyText'] = bodytext
                foi_files.append(temp_data)
            except KeyError:
                pass
        
    json_data['foi_files'] = foi_files


    return json_data

# process_report('2225935', all_results, 'waterland')

In [4]:
def create_dataframe(all_results, org_id):
    all_result_list = []
    
    for k,v in all_results.items():
        if v['_source']['@type'] == 'Report':
            x = process_report(k, all_results, org_id.split('_')[0].lower())
            if x != {}:
                all_result_list.append(x)

    if len(all_result_list) < 100:
        return pd.DataFrame()
    
    df = pd.DataFrame(all_result_list)
    
    df['foi_files_length'] = df['foi_files'].apply(len)

    return df

In [5]:
url = 'https://api.openraadsinformatie.nl/v1/elastic/_cat/indices?v'
response = requests.get(url)
df = pd.read_csv(StringIO(response.text), delim_whitespace=True)
ids = [row['index'] for index, row in df.iterrows() if 'ori_' in row['index']]

In [27]:
display(Counter(classes_list))

Counter({'Moties': 12,
         'Amendementen': 8,
         'Toezeggingen': 6,
         'Brieven aan de raad': 6,
         'Besluitenlijst B&W': 4,
         'RSS': 3,
         'Ingekomen stukken': 3,
         'Schriftelijke vragen': 3,
         'Verslagen raad': 2,
         'Export': 2,
         'Raadsinformatiebrieven': 2,
         'Raadsvragen': 2,
         'Besluitenlijsten college': 1,
         'Schriftelijke vragen en antwoorden': 1,
         'Collegebrieven RSS': 1,
         'Inwerkprogramma': 1,
         'Burgerinitiatieven': 1,
         'Commissie informatiebrieven': 1,
         'export': 1,
         'Besluitenlijsten Politieke avond Westerveld': 1,
         'Raadsvragen (RSS)': 1,
         'Llijst ingekomen stukken': 1,
         'Leesmap coalitievorming': 1,
         'Verslagen': 1,
         'Besluitenlijst': 1,
         'Reglementen': 1,
         'LTA': 1,
         'Verzoek om inlichtingen': 1,
         'Overzicht uitgevoerde moties': 1,
         'Notities ter info': 1,
     

In [26]:
print(orgs)
print(len(orgs))
print(len(classes_list))

['ori_houten_20230705230931', 'ori_roermond_20230706180932', 'ori_midden-delfland_20230706060942', 'ori_westerveld_20230707053943', 'ori_sint_anthonis_20230706193933', 'ori_heemskerk_20230705193926', 'ori_waalre_20230821160619', 'ori_bernheze_20230705063856', 'ori_nederweert_20230821160642', 'ori_vlieland_20230707020940', 'ori_beekdaelen_20230705050937', 'ori_diemen_20230705110910', 'ori_grave_20230705173939', 'ori_oldebroek_20230706120926', 'ori_wassenaar_20230707043942', 'ori_hoorn_20230821161431', 'ori_boxtel_20230705073857', 'ori_stein_20230821161036', 'ori_nijkerk_20230706090923', 'ori_wijchen_20230707070944', 'ori_zeewolde_20230707083946', 'ori_hattem_20230821160810']
22
116


Uit deze analyse blijkt dat er voor slechts 22 gemeenten al een enorme hoeveelheid verschillende termen gebruikt worden (116) er is hier dus geen enkele sprake van uniformiteit. En er zal heel veel normalizering nodig zijn voor het maken van werkbare classificaties. Er zijn wel een aantal vaker voorkomende termen vooral de standaard woorden zoals moties, amendementen, toezeggingen, brieven aan de raad en besluitenlijsten. Maar voor dit soort termen zijn vaak veel variaties aanwezig. 

In [24]:
besluitenlijst_termen = {k: v for k, v in Counter(classes_list).items() if 'besluitenlijst' in k.lower()}
display(besluitenlijst_termen)
print(len(besluitenlijst_termen))

{'Besluitenlijsten college': 1,
 'Besluitenlijsten Politieke avond Westerveld': 1,
 'Besluitenlijst': 1,
 'Besluitenlijsten B&W': 1,
 'Besluitenlijst raadsvergadering': 1,
 'Besluitenlijsten Raadsvergaderingen': 1,
 'Besluitenlijst B&W': 4,
 'Mill – Besluitenlijsten college': 1,
 'Besluitenlijsten': 1,
 'Besluitenlijst B&W Wijchen': 1,
 'Openbaar verslag en besluitenlijst B&W': 1,
 'Besluitenlijst raadscommissies': 1,
 'Besluitenlijst raad': 1}

13


zo zijn er dus 13 verschillende variaties mogelijk voor besluitenlijsten. En dit betekent dus dat waarschijnlijk slechts 16 van de 22 gemeenten besluitenlijsten publiceren, als er geen heel vreemde termen zijn gebruikt. 

In [30]:
besluitenlijst_termen = {k: v for k, v in Counter(classes_list).items() if 'vragen' in k.lower()}
display(besluitenlijst_termen)
print(len(besluitenlijst_termen))

{'Schriftelijke vragen en antwoorden': 1,
 'Raadsvragen (RSS)': 1,
 'Schriftelijke vragen': 3,
 'Schriftelijke vragen (art 39)': 1,
 'Artikel 41 RvO vragen en antwoorden': 1,
 'Schriftelijke vragen (art 33 RvO)': 1,
 'Bestuurlijke vragen': 1,
 'Artikel 36 vragen': 1,
 'Raadsvragen': 2,
 'Vragenhalfuurtje': 1,
 'Schriftelijke vragen (art. 36)': 1}

11


Bijvoorbeeld groeperen op de term vragen levert ook al merkwaardige dingen op. Want er zijn bijvoorbeeld arikel 33, 36, 39 en 41 vragen te vinden. Dus hoe ga je classificeren wat is wat. En wellicht komen al die vragen met de art naam er bij van dezelfde organisatie. Tenminste de eerste 3 neem ik aan!

Verder zijn er ook nog veel vage dingen te vinden die ik hier bijvoorbeeld helemaal niet zou verwachtten. Zoals de nevenfuncties van medewerkers. En zijn er nog veel termen te vinden die voor mij volledig onbekend zijn! Dit lijkt dus een hele grote puzzel te zijn. En dan is dit ongeveer 1/3 van de organisaties die rapporten heeft. Hopenlijk zijn de meetings meer uniform (van wat ik gezien heb denk ik het wel).

In [5]:
id = 'ori_hoorn_20230821161431'
all_results = gather_metadata(id, 10000)
df = create_dataframe(all_results, id)


6154


In [6]:
no_ocr = len([i for i in df.foi_files for a in i if len(''.join([txt for txt in a['foi_bodyText']])) < 20])
all_docs = sum([len(i) for i in df.foi_files])
print(f"{no_ocr} docs zeker niet machineleesbaar")
print(f"{all_docs} docs waarschijnlijk wel machineleesbaar")
print(f"Percentage niet machineleesbaar: {round(no_ocr/all_docs*100,2)}%")

646 docs zeker niet machineleesbaar
3247 docs waarschijnlijk wel machineleesbaar
Percentage niet machineleesbaar: 19.9%


Wat ik over de OCR gelezen dacht te hebben lijkt inderdaad niet te kloppen en ik kan het verder ook nergens terugvinden dus het zal wel niet zo zijn. Het blijkt namelijk dat ongeveer 20 procent van de documenten van Hoorn nog niet machineleesbaar is. 

In [13]:
display(df.isna().sum())

print(df.dc_description.unique())

id                       0
dc_title                 0
dc_source                0
dc_type                  0
foi_classificaties       0
dc_publisher             0
foi_publishedDate        0
dc_date_year             0
foi_files                0
dc_description        1855
foi_files_length         0
dtype: int64

[nan 'Besluitenlijst 10 oktober 2023' 'Besluitenlijst 3 oktober 2023'
 'Besluitenlijst 26 september 2023' 'Besluitenlijst 19 september 2023'
 'Besluitenlijst 12 september 2023' 'Besluitenlijst 5 september 2023'
 'Besluitenlijst 29 augustus 2023' 'Besluitenlijst 22 augustus 2023'
 'Besluitenlijst 8 augustus 2023' 'Besluitenlijst 18 juli 2023'
 'Besluitenlijst 4 juli 2023' 'Besluitenlijst 27 juni 2023'
 'Besluitenlijst 20 juni 2023' 'Besluitenlijst 12 juni 2023'
 'Besluitenlijst 30 mei 2023' 'Besluitenlijst 6 juni 2023'
 'Besluitenlijst 11 juli 2023' 'besluitenlijst 23 mei 2023'
 'Besluitenlijst 16 mei 2023' 'Besluitenlijst 9 mei 2023'
 'Besluitenlijst 2 mei 2023' 'Besluitenlijst 25 april 2023'
 'Besluitenlijst 11 april 2023' 'Besluitenlijst 4 april 2023'
 'Besluitenlijst 28 maart 2023' 'Besluitenlijst 18 april 2023'
 'Besluitenlijst 21 maart 2023' 'Besluitenlijst 14 maart 2023'
 'Besluitenlijst 28 februari 2023' 'Besluitenlijst 7 maart 2023'
 'Besluitenlijst 14 februari 2023' 'Besluiten

In [159]:
# Start a headless Chrome browser
def scrape_ibabs(url):
    driver = webdriver.Chrome()  # You can specify the path to the ChromeDriver executable here

    driver.get(url)
    
    driver.implicitly_wait(1)
    
    # Get the page source (HTML)
    html = driver.page_source
    
    # Close the WebDriver
    driver.quit()
    
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find the dl element containing the data
    data_dl = soup.find('dl', class_='row')
    
    # Initialize empty lists for table headers and rows
    table_headers = []
    table_rows = []
    
    if data_dl:
        # Find all the dt and dd elements within the dl element
        dt_elements = data_dl.find_all('dt')
        dd_elements = data_dl.find_all('dd')

        # Extract the header and data values
        for dt, dd in zip(dt_elements, dd_elements):
            vote_summary = dd.find('div', class_='vote-summary')
            ul = dd.find('ul')
            if vote_summary:
                in_favour_details = []
                against_details = []
                
                try:
                    percentage_vote_in_favour = vote_summary.find('span', class_='vote-summary-bar-in-favour-text').get_text()
                    in_favour = True
                except AttributeError:
                    in_favour = False
                try:
                    percentage_vote_against = vote_summary.find('span', class_='vote-summary-bar-against-text').get_text()
                    against = True
                except AttributeError:
                    against = False

                vote_legend = vote_summary.find('div', class_='vote-summary-legend')

                if vote_legend:

                    if in_favour == True:
                        in_favour_details = vote_legend.find('div', class_='vote-summary-legend-in-favour').text.split('voor', 1)[1].strip().split(', ')
                    if against == True:
                        against_details = vote_legend.find('div', class_='vote-summary-legend-against').text.split('tegen', 1)[1].strip().split(', ')

                    
                    table_headers.append(dt.get_text(strip=True))
                    table_rows.append({'Stemmen voor': in_favour_details, 'Stemmen tegen': against_details})


            elif ul:
                lst = [li.get_text(strip=True) for li in ul.find_all('li')]
    
                table_rows.append(lst)
                table_headers.append(dt.get_text(strip=True))
        
            else:
                table_headers.append(dt.get_text(strip=True))
                if dd.find('a'):
                    a_element = dd.find('a')
                    table_rows.append(f"https://amstelveen.bestuurlijkeinformatie.nl{a_element['href']}")
                else:
                    table_rows.append(dd.get_text(strip=True))
    
    # Create a dictionary from the headers and rows
    data_dict = dict(zip(table_headers, table_rows))
    
    return data_dict

In [160]:
url = 'https://amstelveen.bestuurlijkeinformatie.nl/Reports/Item/49d3f76a-fcea-4c8c-9901-f526dd239700'
scrape_ibabs(url)

{'ID': '709',
 'ID Babs': '',
 'Onderwerp': 'Con Affetto (Met Affectie)',
 'Portefeuillehouder': 'Wethouder Ballegooijen,  van',
 'Indieners': ['CDA', 'ChristenUnie', 'VVD'],
 'Status': 'Aangenomen',
 'Ingediend in': 'Raadsvergadering 18 oktober 2023',
 'Datum ingediend': '18-10-2023',
 'Agendapunt': 'https://amstelveen.bestuurlijkeinformatie.nl/Agenda/Index/1e5a1010-b577-474d-8e36-612d89a05b47#a25b8d40-be4e-4efc-ba4c-7a6f4fb2fcf5',
 'Afgedaan': 'Niet afgedaan',
 'Afgedaan met': '',
 'Einddatum': '',
 'Deadline': '',
 'Opmerkingen griffie': '',
 'Bijlage(s)': ['Motie vreemd aan de orde van de dag CDA. Con Affetto26 KB'],
 'Stemmen': {'Stemmen voor': ['50Plus (1)',
   'Actief voor Amstelveen (2)',
   'Belang van Nederland (1)',
   'Burgerbelangen Amstelveen (5)',
   'CDA (1)',
   'ChristenUnie (1)',
   'D66 (7)',
   'Goed voor Amstelveen (2)',
   'GroenLinks (4)',
   'PvdA (3)',
   'SP (2)',
   'VVD (8)'],
  'Stemmen tegen': []}}